In [175]:
# File location
floc = '/home/dante/SpatialData/spatial_project/data/google/2020_FI_Region_Mobility_Report.csv'

# Assigning dtypes to the to_csv function, as apparently it is very computationally hard
# for Pandas to try and guess dtypes automatically.
dtypes = {
    'country_region_code' : 'object',
    'country_region' : 'object',
    'sub_region_1' : 'object',
    'sub_region_2' : 'object',
    'metro_area' : 'object',
    'iso_3166_2_code' : 'object',
    'census_fips_code' : 'object',
    'place_id' : 'object',
    'date' : 'object',
    'retail_and_recreation_percent_change_from_baseline' : 'float64',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'float64',
    'parks_percent_change_from_baseline' : 'float64',
    'transit_stations_percent_change_from_baseline' : 'float64',
    'workplaces_percent_change_from_baseline' : 'float64',
    'residential_percent_change_from_baseline' : 'float64',
}

# Assigning the correct column names
headers = list(dtypes.keys())

# Telling Pandas what column is a datecolumn (so that it can be parsed)
parse_dates = ['date']

# Reading in file
df_mr = pd.read_csv(floc,header=0,dtype=dtypes,names=headers,parse_dates=parse_dates)

# Sorting values by date and sub_region_2,sub_region_1 as we want to be able to
# look at the data in an intuitive way.
df_mr.sort_values(by=['date','sub_region_2','sub_region_1'],inplace=True)

In [176]:
# Creating a map between the ISO-codes of the different provinces and their ENG/FI/SWE 
# names, for legibility later on.
df_codemap = pd.read_csv('/home/dante/SpatialData/spatial_project/data/google/mob_map.txt',sep=',').iloc[:,0:4]
iso_name_map = dict(zip(df_codemap.iloc[:,0],df_codemap.iloc[:,3]))
df_mr['province'] = df_mr.iso_3166_2_code.map(iso_name_map)


# Dropping unneccesary columns
df_mr.drop(columns=['country_region_code','country_region','census_fips_code','iso_3166_2_code','place_id','metro_area','sub_region_2','sub_region_1'],inplace=True)

# Specifying down our data to the province level
df_mr.dropna(subset=['province'],inplace=True)
df_mr.reset_index(drop=True,inplace=True)

# Casting date in correct format (apparently Pandas doesn't actually read the dtypes correctly)
df_mr['date'] = pd.to_datetime(df_mr.date)

# Adding day of year for gap checking if needed
df_mr['dayofyear'] = df_mr['date'].dt.dayofyear

# Defining the movement data columns of interest
# and filling in NaNs with 0 (i.e no change/information).
# I do not know how sustainable this assumption is...
mvcols = [col for col in df_mr.columns if 'baseline' in col]
values = {col : 0 for col in mvcols}
df_mr.fillna(value=values, inplace=True)

In [177]:
# Checking out the data
df_mr.head(10)

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,province,dayofyear
0,2020-02-15,1.0,2.0,4.0,9.0,-5.0,0.0,Central Finland,46
1,2020-02-15,9.0,22.0,0.0,0.0,-2.0,0.0,Central Ostrobothnia,46
2,2020-02-15,6.0,14.0,17.0,0.0,3.0,0.0,Kainuu,46
3,2020-02-15,5.0,2.0,-17.0,6.0,1.0,0.0,Kymenlaakso,46
4,2020-02-15,17.0,21.0,39.0,56.0,2.0,-1.0,Lapland,46
...,...,...,...,...,...,...,...,...,...
5755,2020-12-31,-10.0,28.0,0.0,-15.0,-59.0,12.0,Southern Ostrobothnia,366
5756,2020-12-31,-16.0,25.0,119.0,-46.0,-51.0,11.0,Southern Savonia,366
5757,2020-12-31,-17.0,15.0,90.0,-38.0,-60.0,13.0,Southwest Finland,366
5758,2020-12-31,-5.0,32.0,0.0,-35.0,-60.0,13.0,Tavastia Proper,366


In [178]:
# Printing the names of the provinces
iso_name_map

{'FI-01': 'Åland',
 'FI-02': 'South Karelia',
 'FI-03': 'Southern Ostrobothnia',
 'FI-04': 'Southern Savonia',
 'FI-05': 'Kainuu',
 'FI-06': 'Tavastia Proper',
 'FI-07': 'Central Ostrobothnia',
 'FI-08': 'Central Finland',
 'FI-09': 'Kymenlaakso',
 'FI-10': 'Lapland',
 'FI-11': 'Pirkanmaa',
 'FI-12': 'Ostrobothnia',
 'FI-13': 'North Karelia',
 'FI-14': 'Northern Ostrobothnia',
 'FI-15': 'Northern Savonia',
 'FI-16': 'Päijänne Tavastia',
 'FI-17': 'Satakunta',
 'FI-18': 'Uusimaa',
 'FI-19': 'Southwest Finland'}

In [180]:
# Printing the province data shapes to make sure our data is comparable...
for idx, prov in enumerate(list(df_mr.province.value_counts().index)):
    print(f"Province {idx} {prov} has shape {df_mr[df_mr.province == prov].shape}")

# based on the above, we will get rid of Åland, because it is not in this dataset,
# nor is it connected geographically to the mainland. Would be interesting to try
# and estimate the connectedness by numbers of ferry crossings between Åland and mainland Finland
# as well as mainland Sweden!!! Just to prove that it matters... Here, the weighting would be by
# day by how many ferry rides across!

Province 0 Central Finland has shape (321, 9)
Province 1 Ostrobothnia has shape (321, 9)
Province 2 Tavastia Proper has shape (321, 9)
Province 3 Southwest Finland has shape (321, 9)
Province 4 Southern Savonia has shape (321, 9)
Province 5 Southern Ostrobothnia has shape (321, 9)
Province 6 Satakunta has shape (321, 9)
Province 7 Päijänne Tavastia has shape (321, 9)
Province 8 Pirkanmaa has shape (321, 9)
Province 9 Northern Savonia has shape (321, 9)
Province 10 Northern Ostrobothnia has shape (321, 9)
Province 11 Lapland has shape (321, 9)
Province 12 Kymenlaakso has shape (321, 9)
Province 13 Uusimaa has shape (321, 9)
Province 14 South Karelia has shape (318, 9)
Province 15 North Karelia has shape (318, 9)
Province 16 Central Ostrobothnia has shape (315, 9)
Province 17 Kainuu has shape (315, 9)


### Creating loop that will make the dataframe just like we want to i.e. in right format:
1) Year-Week Province level

In [183]:
# Creating empty list, so that we can then collect all separate 
# province dataframes into this list when processed by the below
# for - loop...
df_list = []

In [184]:
# This for - loop puts the data in the week - province format.
# Notice that we are using the MEAN movement per week!

# For province in provinces...
for prov in iso_name_map.values():
    
    # Specify the correct province 
    df = df_mr[df_mr.province == prov]
    
    # The below code bins the data according to week commencing MONDAY
    # This code looks complicated, but achieves something quite understanable.
    df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
    df = df.groupby([pd.Grouper(key='date', freq='W-MON')])[mvcols].mean().reset_index().sort_values('date')

    # Creating the column with which we can then merge to the cases data..
    df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.week.astype(str)
    
    # Adding name of province
    df['province'] = prov
    df.reset_index(drop=True) 
    
    # Finally, appending to the list of dfs...
    df_list.append(df)

/tmp/ipykernel_20002/2885164148.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.week.astype(str)
/tmp/ipykernel_20002/2885164148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff']=df.dayofyear.diff()
/tmp/ipykernel_20002/2885164148.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
/tm

In [188]:
# Combining all the province df's into one big dataframe.
combined_mv_df = pd.concat(df_list)

In [189]:
# Having a look at what the end result looks like.
combined_mv_df.head(10)

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Y-W,province
0,2020-02-10,4.333333,3.666667,-7.666667,2.666667,-0.666667,0.333333,2020-7,South Karelia
1,2020-02-17,4.142857,3.857143,-2.714286,3.142857,-5.428571,1.142857,2020-8,South Karelia
2,2020-02-24,-1.857143,-1.000000,7.571429,0.428571,-20.000000,1.857143,2020-9,South Karelia
3,2020-03-02,4.285714,4.000000,2.571429,-2.000000,-0.714286,1.000000,2020-10,South Karelia
4,2020-03-09,-9.000000,3.857143,-2.285714,-19.000000,-7.857143,2.142857,2020-11,South Karelia
...,...,...,...,...,...,...,...,...,...
42,2020-11-30,-15.000000,-2.571429,17.285714,-34.857143,-16.571429,8.000000,2020-49,Southwest Finland
43,2020-12-07,-13.285714,-1.714286,16.285714,-38.285714,-17.714286,8.857143,2020-50,Southwest Finland
44,2020-12-14,-8.142857,3.000000,12.428571,-40.000000,-22.714286,10.285714,2020-51,Southwest Finland
45,2020-12-21,-35.285714,-15.000000,23.714286,-51.571429,-54.571429,13.857143,2020-52,Southwest Finland


In [190]:
# Saving the file as a CSV.
combined_mv_df.to_csv('/home/dante/SpatialData/spatial_project/data/processed/google_mobility_by_week_province.csv',index=False)